This notebook compares analytical and numerical evaluations of alchemical derivatives of a CO molecule up to the third order in KSDFT.  
The results are summerized in the last section.

In [1]:
from pyscf import gto,scf
import numpy as np
import pyscf

pyscf.__version__

'2.2.1'

In [2]:
from FcMole import FcM_like

In [3]:
target_mol = "C 0 0 0; O 0 0 2.1"
dft_functional = "pbe0"  # "lda,vwn"
basis_set = "def2-TZVP"

### Analytical evaluation

In [4]:
%load_ext autoreload
from AP_class import APDFT_perturbator as AP

In [5]:
mol_NN=gto.M(atom=target_mol,unit="Bohr",basis=basis_set)
mf_nn=scf.RKS(mol_NN)
mf_nn.xc = dft_functional
mf_nn.scf()
# mf_nn.conv_tol = 1e-12
# mf_nn.grids.level = 6
# mf_nn.nuc_grad_method().kernel()
ap_nn=AP(mf_nn,sites=[0,1],flag_response_property=True)

converged SCF energy = -113.230628178699


In [6]:
analytical_1st_alchemical_derivative = ap_nn.build_gradient()
analytical_2nd_alchemical_derivative = ap_nn.build_hessian()
analytical_3rd_alchemical_derivative = ap_nn.build_cubic()

Inefficient alchemical force in KS-DFT which calculates the response matrix for nuclear coordinates

In [7]:
inefficient_analytical_alchemical_force = np.zeros((2, 2, 3))
inefficient_analytical_alchemical_force[0] = ap_nn.build_inefficient_alchemical_force(0)
inefficient_analytical_alchemical_force[1] = ap_nn.build_inefficient_alchemical_force(1)

Efficient alchemical force in KS-DFT which calculates the response matrix for nuclear coordinates

In [8]:
efficient_analytical_alchemical_force = np.zeros((2, 2, 3))
efficient_analytical_alchemical_force[0] = ap_nn.build_alchemical_force(0)
efficient_analytical_alchemical_force[1] = ap_nn.build_alchemical_force(1)

Electric dipole derivatives $\partial ^2 E/ \partial Z_I \partial \mathbf{F}$

In [9]:
elec_electric_dipole_gradient_z = ap_nn.build_elec_electric_dipole_gradient(perturb_electron_density='Z')
elec_electric_dipole_gradient_f = ap_nn.build_elec_electric_dipole_gradient(perturb_electron_density='F')

Electric polarizability derivatives $\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F}$

In [10]:
elec_electric_pol_gradient = ap_nn.build_electric_polarizability_gradient()

### Numerical evaluation

In [11]:
%autoreload 2
mol_NN=gto.M(atom=target_mol,unit="Bohr",basis=basis_set)
fc_param = 0.001
fcs1 = [fc_param, 0.0]
fcs2 = [-fc_param, 0.0]
fcs3 = [0.0, fc_param]
fcs4 = [0.0, -fc_param]

##### Efficient numerical alchemical force

In [12]:
fmol1=FcM_like(mol_NN,fcs=fcs1)
fmol2=FcM_like(mol_NN,fcs=fcs2)
fmol3=FcM_like(mol_NN,fcs=fcs3)
fmol4=FcM_like(mol_NN,fcs=fcs4)
mf=scf.RKS(mol_NN)
mf1=scf.RKS(fmol1)
mf2=scf.RKS(fmol2)
mf3=scf.RKS(fmol3)
mf4=scf.RKS(fmol4)
mf.xc = dft_functional
mf1.xc = dft_functional
mf2.xc = dft_functional
mf3.xc = dft_functional
mf4.xc = dft_functional
# mf.conv_tol = 1e-12
# mf1.conv_tol = 1e-12
# mf2.conv_tol = 1e-12
# mf3.conv_tol = 1e-12
# mf4.conv_tol = 1e-12
# mf.grids.level = 6
# mf1.grids.level = 6
# mf2.grids.level = 6
# mf3.grids.level = 6
# mf4.grids.level = 6
# Without dm0=mf.init_guess_by_1e(), some SCFs do not converge.
e=mf.scf(dm0=mf.init_guess_by_1e())
e1=mf1.scf(dm0=mf1.init_guess_by_1e())
e2=mf2.scf(dm0=mf2.init_guess_by_1e())
e3=mf3.scf(dm0=mf3.init_guess_by_1e())
e4=mf4.scf(dm0=mf4.init_guess_by_1e())

converged SCF energy = -113.230628155977


/home/takafumi/program/pyscf/pyscf/gto/mole.py:1213: UserWarning: Function mol.dumps drops attribute with_rinv_at_nucleus because it is not JSON-serializable
  warnings.warn(msg)


converged SCF energy = -113.24532210054
converged SCF energy = -113.215937114715
converged SCF energy = -113.252886091873
converged SCF energy = -113.208373772775


In [13]:
nuc_grad = mf.nuc_grad_method().kernel()
nuc_grad1 = mf1.nuc_grad_method().kernel()
nuc_grad2 = mf2.nuc_grad_method().kernel()
nuc_grad3 = mf3.nuc_grad_method().kernel()
nuc_grad4 = mf4.nuc_grad_method().kernel()

--------------- RKS gradients ---------------
         x                y                z
0 C    -0.0000000000     0.0000000000     0.0310986662
1 O     0.0000000000    -0.0000000000    -0.0311071694
----------------------------------------------
--------------- RKS gradients ---------------
         x                y                z
0 C    -0.0000000000    -0.0000000000     0.0308773925
1 O    -0.0000000000    -0.0000000000    -0.0308858934
----------------------------------------------
--------------- RKS gradients ---------------
         x                y                z
0 C    -0.0000000000     0.0000000000     0.0313199783
1 O     0.0000000000    -0.0000000000    -0.0313284839
----------------------------------------------
--------------- RKS gradients ---------------
         x                y                z
0 C     0.0000000000    -0.0000000000     0.0310518205
1 O    -0.0000000000    -0.0000000000    -0.0310603332
----------------------------------------------
--------

In [14]:
efficient_numerical_alchemical_force = np.zeros((2, 2, 3))
efficient_numerical_alchemical_force[0] = (nuc_grad1 - nuc_grad2) / (2 * fc_param)
efficient_numerical_alchemical_force[1] = (nuc_grad3 - nuc_grad4) / (2 * fc_param)

#### Numerical alchemical derivatives

In [15]:
numerical_1st_alchemical_derivative = np.zeros((2))
numerical_1st_alchemical_derivative[0] = (e1 - e2) / (2 * fc_param)
numerical_1st_alchemical_derivative[1] = (e3 - e4) / (2 * fc_param)

#### Numerical alchemical hardness

In [16]:
ap_nn=AP(mf,sites=[0,1])
ap_nn1=AP(mf1,sites=[0,1],flag_response_property=True,charges_for_center=[6,8])
ap_nn2=AP(mf2,sites=[0,1],flag_response_property=True,charges_for_center=[6,8])
ap_nn3=AP(mf3,sites=[0,1],flag_response_property=True,charges_for_center=[6,8])
ap_nn4=AP(mf4,sites=[0,1],flag_response_property=True,charges_for_center=[6,8])
an = ap_nn.build_gradient()
an1 = ap_nn1.build_gradient()
an2 = ap_nn2.build_gradient()
an3 = ap_nn3.build_gradient()
an4 = ap_nn4.build_gradient()
ann = ap_nn.build_hessian()
ann1 = ap_nn1.build_hessian()
ann2 = ap_nn2.build_hessian()
ann3 = ap_nn3.build_hessian()
ann4 = ap_nn4.build_hessian()
dipole1 = ap_nn1.ref_elec_electric_dipole_moment * -1
dipole2 = ap_nn2.ref_elec_electric_dipole_moment * -1
dipole3 = ap_nn3.ref_elec_electric_dipole_moment * -1
dipole4 = ap_nn4.ref_elec_electric_dipole_moment * -1
pol1 = ap_nn1.ref_elec_electric_polarizability * -1
pol2 = ap_nn2.ref_elec_electric_polarizability * -1
pol3 = ap_nn3.ref_elec_electric_polarizability * -1
pol4 = ap_nn4.ref_elec_electric_polarizability * -1

In [17]:
numerical_2nd_alchemical_derivative = np.zeros((2, 2))
numerical_2nd_alchemical_derivative[0] = (an1 - an2) / (2 * fc_param)
numerical_2nd_alchemical_derivative[1] = (an3 - an4) / (2 * fc_param)

#### Numerical alchemical hardness derivatives

In [18]:
numerical_3rd_alchemical_derivative = np.zeros((2, 2, 2))
numerical_3rd_alchemical_derivative[0, :, :] = (ann1 - ann2) / (2 * fc_param)
numerical_3rd_alchemical_derivative[1, :, :] = (ann3 - ann4) / (2 * fc_param)

#### Inefficient numerical alchemical force

In [19]:
mol_NN1=gto.M(atom="C 0 0 0.001; O 0 0 2.1",unit="Bohr",basis=basis_set)
mol_NN2=gto.M(atom="C 0 0 -0.001; O 0 0 2.1",unit="Bohr",basis=basis_set)
mol_NN3=gto.M(atom="C 0 0 0; O 0 0 2.101",unit="Bohr",basis=basis_set)
mol_NN4=gto.M(atom="C 0 0 0; O 0 0 2.099",unit="Bohr",basis=basis_set)
mf_nn1=scf.RKS(mol_NN1)
mf_nn2=scf.RKS(mol_NN2)
mf_nn3=scf.RKS(mol_NN3)
mf_nn4=scf.RKS(mol_NN4)
mf_nn1.xc = dft_functional
mf_nn2.xc = dft_functional
mf_nn3.xc = dft_functional
mf_nn4.xc = dft_functional
mf_nn1.scf()
mf_nn2.scf()
mf_nn3.scf()
mf_nn4.scf()
ap_nn1=AP(mf_nn1,sites=[0,1])
ap_nn2=AP(mf_nn2,sites=[0,1])
ap_nn3=AP(mf_nn3,sites=[0,1])
ap_nn4=AP(mf_nn4,sites=[0,1])
ad1 = ap_nn1.build_gradient()
ad2 = ap_nn2.build_gradient()
ad3 = ap_nn3.build_gradient()
ad4 = ap_nn4.build_gradient()

converged SCF energy = -113.230596374025
converged SCF energy = -113.230658567281
converged SCF energy = -113.230658567281
converged SCF energy = -113.230596374025


In [20]:
inefficient_numerical_alchemical_force = np.zeros((2, 2))  # only z axis components
inefficient_numerical_alchemical_force[:, 0] = (ad1 - ad2) / (2 * fc_param)
inefficient_numerical_alchemical_force[:, 1] = (ad3 - ad4) / (2 * fc_param)

#### Numerical alchemical electric dipole moment

In [21]:
numer_elec_electric_dipole_gradient = np.zeros((2, 3))
numer_elec_electric_dipole_gradient[0] = (dipole1 - dipole2) / (2 * fc_param)
numer_elec_electric_dipole_gradient[1] = (dipole3 - dipole4) / (2 * fc_param)

#### Numerical alchemical electric polarizability

In [22]:
numer_elec_electric_pol_gradient = np.zeros((2, 3, 3))
numer_elec_electric_pol_gradient[0] = (pol1 - pol2) / (2 * fc_param)
numer_elec_electric_pol_gradient[1] = (pol3 - pol4) / (2 * fc_param)

## Comparison of analytical and numerical evaluations

$ \partial E/\partial Z_i $

In [23]:
print(analytical_1st_alchemical_derivative)

[-14.69249288 -22.25615977]


$ (\partial E/\partial Z_i)_\mathrm{analytical} $ - $ (\partial E/\partial Z_i)_\mathrm{numerical} $

In [24]:
print(analytical_1st_alchemical_derivative - numerical_1st_alchemical_derivative)

[ 3.54426248e-08 -2.25065445e-07]


$\partial^2E/\partial Z_i\partial Z_j$

In [25]:
print(analytical_2nd_alchemical_derivative)

[[-2.90330004  0.45093826]
 [ 0.45093826 -3.55269453]]


$(\partial^2E/\partial Z_i\partial Z_j)_\mathrm{analytical}$ - $(\partial^2E/\partial Z_i\partial Z_j)_\mathrm{numerical}$

In [26]:
print(analytical_2nd_alchemical_derivative - numerical_2nd_alchemical_derivative)

[[-8.95880407e-06  1.97619012e-05]
 [-8.09630862e-07  2.34879672e-06]]


$\partial^3E/\partial Z_i\partial Z_j\partial Z_k$

In [27]:
print(analytical_3rd_alchemical_derivative)

[[[-0.11581771  0.12437604]
  [ 0.12437604  0.08834225]]

 [[ 0.12437604  0.08834225]
  [ 0.08834225 -0.1445824 ]]]


$(\partial^3E/\partial Z_i\partial Z_j\partial Z_k)_\mathrm{analytical}$ - $(\partial^3E/\partial Z_i\partial Z_j\partial Z_k)_\mathrm{numerical}$

In [28]:
print(analytical_3rd_alchemical_derivative - numerical_3rd_alchemical_derivative)

[[[-1.83988841e-06  1.89353258e-07]
  [ 1.89353258e-07  5.97889162e-07]]

 [[-2.15695681e-07  1.43636284e-08]
  [ 1.43636284e-08  1.13489562e-09]]]


$\partial^2E/\partial Z_i\partial R_j$

In [29]:
print(efficient_analytical_alchemical_force)

[[[ 1.25872373e-15 -1.00375460e-15 -2.21297562e-01]
  [-1.24727024e-15  1.05751096e-15  2.21299962e-01]]

 [[-2.97232832e-16 -1.48051554e-17 -4.69293497e-02]
  [ 4.65917479e-16 -3.25902654e-17  4.69198747e-02]]]


$(\partial^2E/\partial Z_i\partial R_j)_\mathrm{analytical}$ - $(\partial^2E/\partial Z_i\partial R_j)_\mathrm{numerical}$

In [30]:
print(efficient_analytical_alchemical_force - efficient_numerical_alchemical_force)

[[[ 6.63093316e-14  3.50538971e-13 -4.68935989e-06]
  [ 1.76611159e-13 -2.17094622e-13  4.71123099e-06]]

 [[-1.53398683e-13  4.25289936e-13 -5.64109706e-07]
  [ 2.91268378e-13  1.41577930e-13  5.65860407e-07]]]


$\partial^2E/\partial Z_i\partial R_j$

In [31]:
print(inefficient_analytical_alchemical_force)

[[[ 1.25487194e-15 -1.00464977e-15 -2.21297562e-01]
  [-1.25301720e-15  1.04360023e-15  2.21299962e-01]]

 [[-2.82138809e-16 -2.34814269e-17 -4.69293493e-02]
  [ 4.50175182e-16  1.02988363e-17  4.69198743e-02]]]


$(\partial^2E/\partial Z_i\partial R_j)_\mathrm{analytical}$ - $(\partial^2E/\partial Z_i\partial R_j)_\mathrm{numerical}$

In [32]:
print(inefficient_analytical_alchemical_force[:, :, 2] - inefficient_numerical_alchemical_force)

[[ 3.98804231e-06 -1.58747751e-06]
 [-7.44098771e-06 -2.03394238e-06]]


$(\partial ^2 E/ \partial Z_I \partial \mathbf{F})_\mathrm{analytical}$

In [33]:
print(elec_electric_dipole_gradient_f)

[[-4.95683077e-14  1.51829940e-14 -6.11496323e-01]
 [-2.14202222e-14  4.43975360e-15  1.22309849e+00]]


In [34]:
print(elec_electric_dipole_gradient_z)

[[-4.97539129e-14  1.53411198e-14 -6.11496323e-01]
 [-2.09748656e-14  4.06829375e-15  1.22309849e+00]]


$(\partial ^2 E/ \partial Z_I \partial \mathbf{F})_\mathrm{analytical}$ - $(\partial ^2 E/ \partial Z_I \partial \mathbf{F})_\mathrm{numerical}$

In [35]:
print(elec_electric_dipole_gradient_f - numer_elec_electric_dipole_gradient)

[[ 3.66345447e-12 -4.33211484e-12 -3.49972763e-05]
 [ 2.33107023e-12  4.54996506e-12 -3.58491558e-06]]


In [36]:
print(elec_electric_dipole_gradient_z - numer_elec_electric_dipole_gradient)

[[ 3.66326887e-12 -4.33195671e-12 -3.49972582e-05]
 [ 2.33151559e-12  4.54959360e-12 -3.58609665e-06]]


$(\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F})_\mathrm{analytical}$

In [37]:
print(elec_electric_pol_gradient)

[[[ 8.02446178e+00  6.34989528e-15 -3.74987920e-13]
  [ 6.34989528e-15  8.02446178e+00  9.80459818e-14]
  [-3.74987920e-13  9.80459818e-14  6.92300681e+00]]

 [[ 4.26378205e+00  2.76883495e-15 -2.36687154e-13]
  [ 2.76883495e-15  4.26378205e+00  6.17838904e-14]
  [-2.36687154e-13  6.17838904e-14  7.73087896e+00]]]


$(\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F})_\mathrm{analytical}$ - $(\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F})_\mathrm{numerical}$

In [38]:
print(elec_electric_pol_gradient - numer_elec_electric_pol_gradient)

[[[ 1.51630452e-05  4.75949222e-12  8.01417120e-11]
  [ 4.75949222e-12  1.51630106e-05 -3.67908078e-11]
  [ 8.01417120e-11 -3.67908078e-11 -5.39926677e-05]]

 [[ 3.76168027e-07 -1.28758183e-11  2.76931870e-11]
  [-1.28758183e-11  3.76202700e-07 -6.42042354e-12]
  [ 2.76931870e-11 -6.42042354e-12 -8.53473703e-06]]]
